Siempre ejecutar esto antes de empezar

Entorno de ejecucion>Ejecutar Todo o Ctrl+F9

In [ ]:
#Santiago Mejia Wilches 
!pip install pyTelegramBotAPI

In [ ]:
import json
import os
import telebot
from telebot import types
user_data = "user_data.json"
if os.path.isfile(user_data):
  print("Usando Local")
else:
  print("Usando Google Drive")
  from google.colab import drive
  drive.mount('/content/drive')
  user_data = "/content/drive/MyDrive/proyectos/proyecto_python/practica/user_data.json"



Usando Google Drive
Mounted at /content/drive


organizador de horarios y transformador a formato de mensaje


---

Posiblemente no funcione si no se corre en mi drive

In [ ]:
def es_admin(id):
  admins=[5400458415,1462809413,5259809689]
  #Santiago Mejia Wilches 5400458415
  #Cristian David Muñoz Camayo 1462809413
  #Katherin Rojas Cadena 5259809689
  for i in admins:
    if i == id:
      return True
  return False
def conseguir_usuarios():
  with open(user_data) as f:
    usuarios = json.load(f)
  f.close()
  return usuarios
def nuevo_usuario(id,horarionuevo):
  usuarios = conseguir_usuarios()
  usuarios.update({id:horarionuevo})
  #usuarios[id]=horarionuevo legacy
  with open(user_data, 'w') as f:
    json.dump(usuarios, f)
  f.close()
def verificar_existencia_horario(id):
  id = str(id)
  usuarios = conseguir_usuarios()
  for name,horarioa in usuarios.items():
    if name == id:
      return True
  return False
def buscar_horario(id):
  id = str(id)
  usuarios = conseguir_usuarios()
  for name,horarioa in usuarios.items():
    if name == id:
      msg = horariomsg(horarioa)
      return msg
  return "No se encontró el horario, puedes usar /nuevo para crear un horario"
# Formato por dia, ej lunes = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] esto significa [6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22] en horario militar
def horariomsg(horario):
  ff=""
  for dia,lista in horario.items():
    diaf=[]
    count=6
    bcount=1
    clase=["",0,0]
    horatxt=""
    # clase=["Nombre de la clase",duracion,hora de inicio]
    for hora in lista:
      if hora == 0:
        if horatxt!="":
          clase=[horatxt,bcount,count-bcount]
          clasetxt="Inicio: "+str(clase[2])+" / Nombre: "+clase[0]+" / Duración: "+str(clase[1])+" horas"
          diaf.append(clasetxt)
          horatxt=""
      elif hora == horatxt:
        bcount+=1
      else:
        if horatxt!="":
          clase=[horatxt,bcount,count-bcount]
          clasetxt="Inicio: "+str(clase[2])+" / Nombre: "+clase[0]+" / Duración: "+str(clase[1])+" horas"
          diaf.append(clasetxt)
          bcount=1
          horatxt=hora
        else:
          horatxt=hora
          bcount=1
      count+=1
    ff=ff+"---------------------------\n"
    ff=ff+dia.title()+":"+"\n"
    if diaf==[]:
      ff=ff+"Este día no tienes clase\n"
    else:
      for i in diaf:
        ff=ff+i+"\n"
    ff=ff+"---------------------------\n"
  #print(ff)
  return ff


In [ ]:
def teclado_siono():
  markup = types.ReplyKeyboardMarkup(one_time_keyboard=True)
  itembtna = types.KeyboardButton('Si')
  itembtnv = types.KeyboardButton('No')
  markup.row(itembtna, itembtnv)
  return markup

In [ ]:
def crear_horario(message):
  crearhorario_primero.si=message
  reset()
  crear_horario.counter+=1
  if crear_horario.counter < 5 and crear_horario.counter != 5:
    txt = message.text.title()
    if txt == "Si":
      horario.lista=[]
      diadar = "¿Tienes clase el "+dias_semana[crear_horario.counter]+"?"
      msg = bot.send_message(message.chat.id,diadar, reply_markup = teclado_siono())
      bot.register_next_step_handler(msg,crear_horas)
      #bot.register_for_reply(msg,horas(message.chat.id,message.text))
    elif txt == "No":
      msg = bot.send_message(message.chat.id,"Okay")
      crear_horario(crearhorario_primero.si)
    else:
      crear_horario.counter-=1
      msg = bot.reply_to(message.chat.id,"Responde con Si o No")
      bot.register_next_step_handler(message,crear_horario)
  else:
    nuevo_usuario(message.from_user.id,horario_nuevo)
    bot.send_message(message.chat.id,"Tu horario fue creado\nUsa /horario para verlo", reply_markup = types.ReplyKeyboardRemove())
def crear_horas(message):
  txt = message.text.title()
  if txt == "Si":
    msg = bot.send_message(message.chat.id,"¿A qué hora es tu clase? (formato 24h)", reply_markup = types.ReplyKeyboardRemove())
    bot.register_next_step_handler(msg, horas)
  elif txt == "No":
    msg = bot.send_message(message.chat.id,"Okay")
    reset()
    horario_nuevo_func()
  else:
    crear_horario.counter-=1
    msg = bot.reply_to(message.chat.id, "Responde con Si o No")
    bot.register_next_step_handler(message, crear_horario)

def horas(message):
  txt = message.text.title()
  try:
    horario.hora = int(txt)
    msg = bot.send_message(message.chat.id,"¿Cuánto dura esta clase?", reply_markup = types.ReplyKeyboardRemove())
    bot.register_next_step_handler(msg, duras)
  except:
    bot.send_message(message.chat.id,"Ingresa un valor numérico válido, usa un solo número", reply_markup = types.ReplyKeyboardRemove())
    crear_horas(crearhorario_primero.si)

def duras(message):
  txt = message.text.title()
  try:
    horario.dura = int(txt)
    msg = bot.send_message(message.chat.id,"¿Con qué nombre quieres guardar esta clase?\nIntenta usar nombres cortos para que se vea mejor")
    bot.register_next_step_handler(msg, final_dia)
  except:
    bot.send_message(message.chat.id,"Ingresa un valor numérico válido, usa un solo número", reply_markup = teclado_siono())
    horas(crearhorario_primero.si)

def final_dia(message):
  horario.nombre = message.text
  debug()
  horario()
  msg = bot.send_message(message.chat.id,"¿Tienes otra clase este día?", reply_markup = teclado_siono())
  bot.register_next_step_handler(msg, dia_final_check)

def dia_final_check(message):
  txt = message.text.title()
  if txt == "Si":
    crear_horas(crearhorario_primero.si)
  elif txt == "No":
    horario_nuevo_func()

def debug():
  print("horario.nombre:",horario.hora)
  print("horario.dura:",horario.dura)
  print("horario.nombre:",horario.nombre)

def horario():
  horario.lista = [0,0,""]
  #print("\naqui")
  horario.lista[0] = horario.hora
  #print("\naqui")
  horario.lista[1] = horario.dura
  #print("\naqui")
  horario.lista[2] = horario.nombre
  #print("\naqui")
  #horario.lista = [horario.hora,horario.dura,horario.nombre]
  for i in range(6,23):
    if horario.lista[0] == i:
      b = horario.lista[1]
      b = (b + i)- 6
      c = i - 6
      print("B",b)
      print("C",c)
      for a in range(c,b):
        crear_horario.default_dia[a] = horario.lista[2]
  print(crear_horario.default_dia)

horario_nuevo = {}
crear_horario.default_dia = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
dias_semana = ["lunes","martes","miércoles","jueves","viernes"]

def reset():
  crear_horario.default_dia = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  print("Reset")
def horario_nuevo_func():
  horario_nuevo[dias_semana[crear_horario.counter]] = crear_horario.default_dia
  crear_horario(crearhorario_primero.si)
  print(horario_nuevo)
#horario.hora = 8
#horario.dura = 2
#horario.nombre = "Mecanica"
#horario.lista = [horario.hora,horario.dura,horario.nombre]

#horario()
crear_horario.counter=-1

Telegram Bot

In [ ]:
#t.me/Cosmi_bot
bot = telebot.TeleBot("", parse_mode=None) # You can set parse_mode by default. HTML or MARKDOWN

@bot.message_handler(commands=['start'])
def send_welcome(message):
	#print(message.from_user.first_name)
	msg = "Hola " + str(message.from_user.first_name) + ", puedes usar /help para ver la lista de comandos disponibles"
	bot.reply_to(message, msg)

@bot.message_handler(commands=["horario","Horario","HORARIO"])
def echo_all(message):
	bot.reply_to(message, buscar_horario(message.from_user.id), reply_markup = types.ReplyKeyboardRemove())

@bot.message_handler(commands=["help","Help","HELP"])
def echo_all(message):
	bot.reply_to(message,"/start Inicia el bot\n/help Muestra una lista de comandos disponibles\n/horario Muestra el horario guardado\n/nuevo Crea o reemplaza un horario existente")

@bot.message_handler(commands=["nuevo","Nuevo","NUEVO"])
def echo_all(message):
	m = message.from_user.id
	print(m,"esta creando un horario")
	if verificar_existencia_horario(m):
		bot.reply_to(message, "Vamos a crear tu nuevo horario")
		msg = bot.send_message(message.chat.id, "Ya tienes un horario ¿Quieres reemplazarlo?", reply_markup = teclado_siono())
		bot.register_next_step_handler(msg, crearhorario_primero)
	else:
		bot.reply_to(message, "Vamos a crear tu nuevo horario")
		msg = bot.send_message(message.chat.id,"Este proceso puede llevar un rato (5min, aprox) ¿Seguro que quieres iniciar ahora?", reply_markup = teclado_siono())
		crear_horario.counter=-1
		bot.register_next_step_handler(msg, crear_horario)
def crearhorario_primero(message):
	txt = message.text.title()
	if txt == "Si":
		#crearhorario_primero.si=message
		msg = bot.send_message(message.chat.id,"Este proceso puede llevar un rato (5min, aprox) ¿Seguro que quieres iniciar ahora?", reply_markup = teclado_siono())
		crear_horario.counter=-1
		bot.register_next_step_handler(msg, crear_horario)
	elif txt == "No":
		msg = bot.send_message(message.chat.id,"Okay")
	else:
		msg = bot.reply_to(message.chat.id, "Responde con Si o No")
		bot.register_next_step_handler(message, crearhorario_primero)

#comandos admin

@bot.message_handler(commands=["id"])
def echo_all(message):
	if es_admin(message.from_user.id):
		bot.reply_to(message, message.from_user.id)
	else:
		pass

@bot.message_handler(commands=["test"])
def echo_all(message):
	if es_admin(message.from_user.id):
		print(message.id)
		print(message.chat.id)
	else:
		pass

@bot.message_handler(commands=["test1"])
def echo_all(message):
	if es_admin(message.from_user.id):
		a = bot.copy_message(chat_id = message.chat.id,from_chat_id = message.chat.id,message_id=716)
		print(a)
	else:
		pass

@bot.message_handler(commands=["removemarkup"])
def echo_all(message):
	if es_admin(message.from_user.id):
		bot.send_message(message.chat.id, "Markup Removed", reply_markup = types.ReplyKeyboardRemove())
	else:
		pass

@bot.message_handler(commands=["stop"])
def echo_all(message):
	if es_admin(message.from_user.id):
		bot.stop_polling()
	else:
		pass

bot.infinity_polling()